In [1]:
import scapy.all as scapy

In [12]:
dx = scapy.rdpcap('./pcap.pcap')

In [44]:
v1 = '192.168.221.105'
v2 = '192.168.221.91'

import binascii
start = binascii.unhexlify('e766e65ae8509d6833d73a37d1ec4ad8189919a5402f801531e936d132e5df42f22f99c8db9ffe673a0352c58d7999c4')
started = False
from collections import defaultdict
data = dict(v2c= bytearray(), c2v=bytearray())

cnta = 0
for pkt in dx:
  ip = pkt[scapy.IP]
  between = set([ip.src, ip.dst])
  if (v1 not in between) or (v2 not in between): continue
  tcp = pkt[scapy.TCP]
  payload = bytes(pkt.payload)
  if not started:
    if len(payload) < len(start): continue
    if payload[-len(start):] != start: continue
    started = True
  
  payload = b'\x00'*0xc + payload
  typ = 'v2c'
  if v2 == ip.src:
    s = payload[0xa8:]
    
    if len(tcp.payload) == 1460:
      if cnta !=0 : s = bytes(tcp.payload)
      cnta += 1
    elif cnta!=0:
      s = bytes(tcp.payload)
      cnta = 0
    if len(s)< 4: continue
    typ = 'c2v'
  else:
    s = payload[0x88:]
  if len(s) != 0x30  and (s[2:4] != b'\x00\x8f' and len(s) < 93): continue
  print(len(s), pkt.time, s[:4], len(payload))
  data[typ] += s
    


48 1533914614.586462 b'\xe7f\xe6Z' 216
48 1533914614.592582 b'_\xa5)@' 184
107 1533914616.599037 b'k\x00\x00\x8f' 275
81 1533914616.600466 b'Q\x00\x00\x8f' 217
84 1533914618.600312 b'T\x00\x00\x8f' 252
206 1533914618.600784 b'\xce\x00\x00\x8f' 342
84 1533914623.612401 b'T\x00\x00\x8f' 252
203 1533914623.617886 b'\xcb\x00\x00\x8f' 339
84 1533914627.613389 b'T\x00\x00\x8f' 252
86 1533914627.634219 b'V\x00\x00\x8f' 222
191 1533914627.735362 b'\xbf\x00\x00\x8f' 327
93 1533914634.621904 b']\x00\x00\x8f' 261
84 1533914634.624136 b'T\x00\x00\x8f' 220
97 1533914634.723806 b'a\x00\x00\x8f' 233
90 1533914637.623945 b'Z\x00\x00\x8f' 258
84 1533914637.626659 b'T\x00\x00\x8f' 220
381 1533914637.672814 b'}\x01\x00\x8f' 517
97 1533914641.629495 b'a\x00\x00\x8f' 265
84 1533914641.631835 b'T\x00\x00\x8f' 220
101 1533914641.712784 b'e\x00\x00\x8f' 237
90 1533914645.635067 b'Z\x00\x00\x8f' 258
84 1533914645.643271 b'T\x00\x00\x8f' 220
325 1533914645.659874 b'E\x01\x00\x8f' 461
120 1533914649.637483 b'x\x

In [45]:
import pickle
pickle.dump(data, open('./conn2.transfer.pickle', 'wb'))